In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost') # 127.0.0.1

In [4]:
client.HOST

'localhost'

In [5]:
client.PORT

27017

In [8]:
client.server_info()

{'allocator': 'system',
 'bits': 64,
 'buildEnvironment': {'cc': 'gcc: Apple LLVM version 8.1.0 (clang-802.0.42)',
  'ccflags': '-mmacosx-version-min=10.10 -fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-unused-private-field -Wno-deprecated-declarations -Wno-tautological-constant-out-of-range-compare -Wno-unused-const-variable -Wno-missing-braces -Wno-inconsistent-missing-override -Wno-potentially-evaluated-expression -Wno-exceptions -fstack-protector-strong -fno-builtin-memcmp',
  'cxx': 'g++: Apple LLVM version 8.1.0 (clang-802.0.42)',
  'cxxflags': '-Woverloaded-virtual -Werror=unused-result -Wpessimizing-move -Wredundant-move -Wno-undefined-var-template -stdlib=libc++ -std=c++14',
  'distarch': 'x86_64',
  'distmod': '',
  'linkflags': '-mmacosx-version-min=10.10 -Wl,-bind_at_load -Wl,-fatal_warnings -fstack-protector-strong -stdlib=libc++',
  'target

In [9]:
client.close() # 서버랑 연결을 끊음

In [10]:
client.list_database_names()

['admin', 'config', 'local']

In [11]:
list_db = client.list_databases()

In [12]:
list_db

In [13]:
list(list_db)

[{'empty': False, 'name': 'admin', 'sizeOnDisk': 8192.0},
 {'empty': False, 'name': 'config', 'sizeOnDisk': 12288.0},
 {'empty': False, 'name': 'local', 'sizeOnDisk': 8192.0}]

In [14]:
list(list_db) # 두번째는 아무런 값이 안 뜬다.   'command_cursor'의 개념 !

[]

In [15]:
for i in list_db :
    print(i)
    print(type(i))

In [16]:
type(client.get_database('admin'))

pymongo.database.Database

In [17]:
admin = client.get_database('admin') # admin : use admin 하여 사용할 수 있는 명령어들을 실행시키는 변수

In [18]:
admin.list_collection_names()

['system.version']

In [19]:
test = client.get_database('test')
test

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

In [20]:
test2 = client.test
test2

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

In [22]:
item = test.get_collection('item')
item

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'item')

In [23]:
test2 = test.item

In [24]:
test['item']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'item')

In [25]:
newitem = {'name' : 'item1',
          'price' : 10000,
          'ratio' : 4.5}

In [26]:
item.insert_one(newitem)

In [32]:
result = item.find({})
result

In [39]:
print(list(result))
print(type(result))

[]
<class 'pymongo.cursor.Cursor'>


In [40]:
result = item.find({})

i = list(result)
i[0]

{'_id': ObjectId('5b4eaa1ea56e0d7c018462e6'),
 'name': 'item1',
 'price': 10000,
 'ratio': 4.5}

In [41]:
newitem2 = {
    'name' : 'item2',
    'price' : float(10000),
    'ratio' : 4.1
}

In [42]:
item.insert_one(newitem2)

---

In [44]:
from pymongo import MongoClient
client = MongoClient('localhost', username = "root", password = '1q2w3e4r', authsource = "admin")
client.list_database_names()

OperationFailure: Authentication failed.

In [46]:
from pymongo import MongoClient
client = MongoClient("mongodb://127.0.0.1:27017")
client.list_database_names()

['admin', 'config', 'local', 'test']

---

In [47]:
client.sns.get_collection('user') # 새로운 컬렉션 생성

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'sns'), 'user')

In [48]:
client.sns.get_collection('feed') # 새로운 컬랙션 생성

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'sns'), 'feed')

아래와 같이 함수로 정의하면 더욱 쉽게 insert 할 수 있다.

In [49]:
def create_user(user_id, user_pwd, name, email) :
    new_user = {'_id' : user_id,
               'pwd' : user_pwd,
               'name' : name,
               'email' : email,
               'address' : [],
               'profile' : "",
               'friends' : []} 
    return client.sns.user.insert_one(new_user)

In [50]:
create_user('efgh','1q2w3e4r','Jim','jim@email.com')

In [51]:
def add_address(user_id, address, info, tel) :
    new_address = {'address' : address,
                  'info' : info,
                  'tel' : tel}
    client.sns.user.update_one(
    {'_id' : user_id},
    {'$addToSet' : {'address' : new_address}}) # 리스트에 새로이 인자를 추가

In [52]:
add_address('abcd', 'Seoul Korea', 'Seoul', '02-0000-0000')

In [53]:
add_address('abcd', 'New York United States', 'New York' , '0+000-000-0000')

In [55]:
add_address('abcd', 'Beijing China', 'Beijing' , '0+000-000-0000')

In [57]:
def delete_address(user_id, address) :
    client.sns.user.update_one(
    {'_id' : user_id}, 
        {'$pull' : {'address.address' : address}})

In [58]:
delete_address('abcd', 'Beijing China')

In [59]:
validator = { # 스키마를 적용시킴
    '$JsonSchema' : {
        'bsonType' : 'object',
        'required' : ['name', 'member'],
        'properties' : {
            'name' : {
                'bsonType' : 'string',
                'description' : 'Name must be string'
            },
            'member' : {
                'bsonType' : 'array',
                'description' : 'Member must be array'
            }
        }
    }
}